In [31]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
meta_data = pd.read_csv("Meta_Data.csv")

In [4]:
meta_data['action'].value_counts(dropna=False)

2_Fingers_Right    102
Shake_Hand         102
2_Fingers_Left      95
1_Finger_Click      94
Name: action, dtype: int64

In [5]:
file_path = meta_data['file_path'].tolist()
action = meta_data['action'].tolist()

In [9]:
file_path

['./Train_Data\\1_Finger_Click\\Video_Test_10 (2).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (3).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (4).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_10.mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (2).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (3).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_11.mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (2).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (3).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (4).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_12.mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (2).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (3).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_13.mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (2).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (3).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (4).mp4',
 './Train_Data\\1_Finger_Click\\Video_Test_14.mp4',
 './Train_Da

In [6]:
mp_hands = mp.solutions.hands # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [35]:
zero_list = [0] * 63

#for file in tqdm(file_path[:1]):
for file in tqdm(file_path):
    
    result = []
    cap = cv2.VideoCapture( file )
    
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    with mp_hands.Hands( max_num_hands = 1,
                        min_detection_confidence=0.5, 
                        min_tracking_confidence=0.5) as hands:
        frame = 0
        while True:
            success, image = cap.read()

            if success == False:
                cap.release()
                break
            
            frame = frame+1
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = hands.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR

            if results.multi_hand_landmarks:
                
                one_frame_result = []
                
                for hand_landmarks in results.multi_hand_landmarks:
                    for ids, landmrk in enumerate(hand_landmarks.landmark):
                        #print(landmrk.x)
                        one_frame_result.append( landmrk.x )
                        one_frame_result.append( landmrk.y )
                        one_frame_result.append( landmrk.z )
                        
                result.append( one_frame_result )
                
            else:
                result.append( zero_list )
                
        a = pd.DataFrame(result)
        csv_file_name = file.replace("mp4" , "csv")
        a.to_csv(csv_file_name , index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 393/393 [10:37<00:00,  1.62s/it]


In [37]:
csv_file_list = []

for file in file_path:
    csv_file_list.append( file.replace("mp4" , "csv") )

In [38]:
csv_file_list

['./Train_Data\\1_Finger_Click\\Video_Test_10 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14.csv',
 './Train_Da

In [39]:
meta_data['csv_file_path'] = csv_file_list

In [40]:
meta_data

,file_path,action,csv_file_path
0,./Train_Data\1_Finger_Click\Video_Test_10 (2).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (2).csv
1,./Train_Data\1_Finger_Click\Video_Test_10 (3).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (3).csv
2,./Train_Data\1_Finger_Click\Video_Test_10 (4).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (4).csv
3,./Train_Data\1_Finger_Click\Video_Test_10.mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10.csv
4,./Train_Data\1_Finger_Click\Video_Test_11 (2).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_11 (2).csv
...,...,...,...
388,./Train_Data\Shake_Hand\Video_Test_8.mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_8.csv
389,./Train_Data\Shake_Hand\Video_Test_9 (2).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (2).csv
390,./Train_Data\Shake_Hand\Video_Test_9 (3).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (3).csv
391,./Train_Data\Shake_Hand\Video_Test_9 (4).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (4).csv


In [41]:
meta_data.to_csv("Meta_Data_211229.csv" , index=False)